# Module 6: Building Data APIs with FastAPI

## Learning Objectives
- Master FastAPI fundamentals
- Learn API design and documentation
- Practice data validation with Pydantic
- Understand authentication and security

## Topics Covered
- FastAPI setup and routing
- API design and documentation
- Data validation with Pydantic
- Authentication and security
- Database integration with FastAPI

## Time Estimate: 3-4 weeks
## Success Criteria: Build a complete REST API with authentication and documentation

---

## Section 1: FastAPI Setup and Basic Routing

FastAPI is a modern, fast web framework for building APIs with Python 3.7+ based on standard Python type hints. It's designed to be easy to use and learn, fast to code, and ready for production.


In [ ]:
# FastAPI Basic Setup and Routing

from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel, EmailStr
from typing import List, Optional
from datetime import datetime
import uvicorn

# Create FastAPI instance
app = FastAPI(
    title="Learning API",
    description="A simple API for learning FastAPI concepts",
    version="1.0.0"
)

# Pydantic models for request/response validation
class UserBase(BaseModel):
    username: str
    email: EmailStr
    first_name: str
    last_name: str

class UserCreate(UserBase):
    pass

class UserResponse(UserBase):
    id: int
    created_at: datetime
    
    class Config:
        from_attributes = True

class PostBase(BaseModel):
    title: str
    content: str

class PostCreate(PostBase):
    author_id: int

class PostResponse(PostBase):
    id: int
    author_id: int
    created_at: datetime
    
    class Config:
        from_attributes = True

# In-memory storage (for demo purposes)
users_db = []
posts_db = []
next_user_id = 1
next_post_id = 1

# Basic routes
@app.get("/")
async def root():
    """Root endpoint"""
    return {"message": "Welcome to the Learning API!", "version": "1.0.0"}

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "timestamp": datetime.utcnow()}

# User routes
@app.post("/users/", response_model=UserResponse)
async def create_user(user: UserCreate):
    """Create a new user"""
    global next_user_id
    
    # Check if username or email already exists
    for existing_user in users_db:
        if existing_user["username"] == user.username:
            raise HTTPException(status_code=400, detail="Username already registered")
        if existing_user["email"] == user.email:
            raise HTTPException(status_code=400, detail="Email already registered")
    
    # Create new user
    new_user = {
        "id": next_user_id,
        "username": user.username,
        "email": user.email,
        "first_name": user.first_name,
        "last_name": user.last_name,
        "created_at": datetime.utcnow()
    }
    
    users_db.append(new_user)
    next_user_id += 1
    
    return new_user

@app.get("/users/", response_model=List[UserResponse])
async def get_users():
    """Get all users"""
    return users_db

@app.get("/users/{user_id}", response_model=UserResponse)
async def get_user(user_id: int):
    """Get a specific user by ID"""
    for user in users_db:
        if user["id"] == user_id:
            return user
    raise HTTPException(status_code=404, detail="User not found")

# Post routes
@app.post("/posts/", response_model=PostResponse)
async def create_post(post: PostCreate):
    """Create a new post"""
    global next_post_id
    
    # Check if author exists
    author_exists = any(user["id"] == post.author_id for user in users_db)
    if not author_exists:
        raise HTTPException(status_code=404, detail="Author not found")
    
    # Create new post
    new_post = {
        "id": next_post_id,
        "title": post.title,
        "content": post.content,
        "author_id": post.author_id,
        "created_at": datetime.utcnow()
    }
    
    posts_db.append(new_post)
    next_post_id += 1
    
    return new_post

@app.get("/posts/", response_model=List[PostResponse])
async def get_posts():
    """Get all posts"""
    return posts_db

@app.get("/posts/{post_id}", response_model=PostResponse)
async def get_post(post_id: int):
    """Get a specific post by ID"""
    for post in posts_db:
        if post["id"] == post_id:
            return post
    raise HTTPException(status_code=404, detail="Post not found")

@app.get("/users/{user_id}/posts", response_model=List[PostResponse])
async def get_user_posts(user_id: int):
    """Get all posts by a specific user"""
    # Check if user exists
    user_exists = any(user["id"] == user_id for user in users_db)
    if not user_exists:
        raise HTTPException(status_code=404, detail="User not found")
    
    # Filter posts by author
    user_posts = [post for post in posts_db if post["author_id"] == user_id]
    return user_posts

print("✅ FastAPI application created!")
print("✅ Routes defined:")
print("- GET / (root)")
print("- GET /health (health check)")
print("- POST /users/ (create user)")
print("- GET /users/ (get all users)")
print("- GET /users/{user_id} (get user by ID)")
print("- POST /posts/ (create post)")
print("- GET /posts/ (get all posts)")
print("- GET /posts/{post_id} (get post by ID)")
print("- GET /users/{user_id}/posts (get user's posts)")

print("\nTo run the API server, use:")
print("uvicorn fastapi_apis:app --reload --host 0.0.0.0 --port 8000")
